In [2]:
#Dependencies
import pandas as pd
import csv
import matplotlib
import seaborn
import os
import numpy as np
import time
import json
from config import wolf_api_key
from pprint import pprint
import requests


In [2]:
#import csv into dataframe and preview
cities_df = pd.read_csv("500_Cities_original.csv", encoding="ISO-8859-1")

In [3]:
#Keep only city-level rows
cities_df = cities_df.loc[cities_df["GeographicLevel"] == "City"]

In [4]:
#keep only age-adjusted data values
#Keep only city-level rows
cities_df = cities_df.loc[cities_df["DataValueTypeID"] == "AgeAdjPrv"]
cities_df.head()

,Year,StateAbbr,StateDesc,CityName,GeographicLevel,DataSource,Category,UniqueID,Measure,Data_Value_Unit,...,High_Confidence_Limit,Data_Value_Footnote_Symbol,Data_Value_Footnote,PopulationCount,GeoLocation,CategoryID,MeasureId,CityFIPS,TractFIPS,Short_Question_Text
56,2016,AL,Alabama,Birmingham,City,BRFSS,Prevention,0107000,Current lack of health insurance among adults ...,%,...,19.9,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",PREVENT,ACCESS2,107000.0,NaN,Health Insurance
156,2016,AL,Alabama,Birmingham,City,BRFSS,Health Outcomes,0107000,Arthritis among adults aged >=18 Years,%,...,29.4,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",HLTHOUT,ARTHRITIS,107000.0,NaN,Arthritis
256,2016,AL,Alabama,Birmingham,City,BRFSS,Unhealthy Behaviors,0107000,Binge drinking among adults aged >=18 Years,%,...,13.1,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",UNHBEH,BINGE,107000.0,NaN,Binge Drinking
359,2015,AL,Alabama,Birmingham,City,BRFSS,Health Outcomes,0107000,High blood pressure among adults aged >=18 Years,%,...,44.2,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",HLTHOUT,BPHIGH,107000.0,NaN,High Blood Pressure
459,2015,AL,Alabama,Birmingham,City,BRFSS,Prevention,0107000,Taking medicine for high blood pressure contro...,%,...,70.2,NaN,NaN,212237,"(33.5275663773, -86.7988174678)",PREVENT,BPMED,107000.0,NaN,Taking BP Medication


In [ ]:
#trim down columns
cities_cleaned_df = cities_df.loc[:, ["Year", "StateDesc", "CityName",
                                    "Category", "Measure", "DataValueTypeID", "Data_Value", "PopulationCount",
                                      "Count_of_Individuals", "GeoLocation", "MeasureId", "CityFIPS",
                                      "Short_Question_Text"]]
cities_cleaned_df.head(6)

In [31]:
just_cities = cities_df.loc[:, ["StateDesc","CityName","PopulationCount","GeoLocation","CityFIPS"]]
just_cities = just_cities.drop_duplicates()

# Make a small dataframe to play around with
just_cities_head = just_cities.head(3)
just_cities.head()

,StateDesc,CityName,PopulationCount,GeoLocation,CityFIPS
56,Alabama,Birmingham,212237,"(33.5275663773, -86.7988174678)",107000.0
2851,Alabama,Hoover,81619,"(33.3767602729, -86.8051937568)",135896.0
3713,Alabama,Huntsville,180105,"(34.6989692671, -86.6387042882)",137000.0
5549,Alabama,Mobile,195111,"(30.6776248648, -88.1184482714)",150000.0
7779,Alabama,Montgomery,205764,"(32.3472645333, -86.2677059552)",151000.0


# Add Weather

In [13]:
# #%%
# # Dependencies
# import requests
# import json
# from pprint import pprint

# # Get API Key
# from config import wolfram_api_key

# url = "https://api.wolframalpha.com/v2/query?format=plaintext&output=JSON&input="

# #%%
# # City and query
# city = "Raleigh+NC"
# query = "average+temperature+for+" + city + "+past+ten+years"

# # Build query URL
# query_url = url + query + "&appid=" + wolfram_api_key
# # Do not print to GitHub
# # print(query_url)

# #%%
# # Request Weather
# weather = requests.get(query_url).json()

# # Print the json
# # print(weather)

# #%%
# # Print the json (pretty printed)
# # print(json.dumps(weather, indent=4, sort_keys=True))

# #%%
# # Get Average Temp
# avg_temp = weather["queryresult"]["pods"][1]["subpods"][0]["plaintext"]
# # print(avg_temp)
# temp_string = weather["queryresult"]["pods"][2]["subpods"][0]["plaintext"]
# print(temp_string)

In [17]:
# # Split the String

# # Overall Average Temp
# avg_temp_clean = int(avg_temp.split(" °F")[0])
# print(f"The avg_temp_clean variable = {avg_temp_clean}")

# # Historic Low Temp
# temp_low = temp_string.split(" | ")[2]
# temp_low = temp_low.split("low: ")[1]
# temp_low = int(temp_low.split(" °F")[0])
# print(f"The temp_low variable = {temp_low}")
# # print("")

# # Average High Temp
# temp_avghigh = temp_string.split(" | ")[4]
# temp_avghigh = int(temp_avghigh.split(" °F")[0])
# print(f"The temp_avghigh variable = {temp_avghigh}")
# # print("")

# # Average Low Temp
# temp_avglow = temp_string.split(" | ")[5]
# temp_avglow = int(temp_avglow.split(" °F")[0])
# print(f"the temp_avglow variable = {temp_avglow}")
# # print("")

# # Historic High Temp
# temp_high = temp_string.split(" | ")[6]
# temp_high = temp_high.split("high: ")[1]
# temp_high = int(temp_high.split(" °F")[0])
# print(f"The temp_high variable = {temp_high}")
# # print("")

# # Temperature Range
# average_temp_range = temp_avghigh - temp_avglow
# print(f"The range in average temp = {average_temp_range}")

The avg_temp_clean variable = 59
The temp_low variable = 4
The temp_avghigh variable = 83
the temp_avglow variable = 39
The temp_high variable = 105
The range in average temp = 44


In [37]:
# Get Weather Loop

# Build partial query URL
url = "https://api.wolframalpha.com/v2/query?format=plaintext&output=JSON&input="


print("Beginning Data Retrieval")
print("------------------------")

row_count = 1

avg_temp_clean = []
temp_low_list = []
temp_avghigh_list = []
temp_avglow_list = []
temp_high_list = []
# average_temp_range_list = []

for index, row in just_cities_head.iterrows():
    city_query = row['CityName']
    state_query = row['StateDesc']
    print(f"This is the city query {city_query}, {state_query}")
    try:
        
        ten_year_query = "average+temperature+for+" + city_query.replace(" ","+") + "+" + state_query.replace(" ","+") + "+past+ten+years"

        # Build query URL
        query_url = url + ten_year_query + "&appid=" + wolfram_api_key
                       
        # Get Response
        response = requests.get(query_url).json()
                       
        # Obtain, then Append Overall Average Temp
        avg_temp = response["queryresult"]["pods"][1]["subpods"][0]["plaintext"]
        avg_temp_clean.append(int(avg_temp.split(" °F")[0]))

        # Get Temp String (used for all other variables below)
        temp_string = response["queryresult"]["pods"][2]["subpods"][0]["plaintext"]
        
        # Historic Low Temp
        temp_low = temp_string.split(" | ")[2]
        temp_low = temp_low.split("low: ")[1]
        temp_low_list.append(int(temp_low.split(" °F")[0]))

        # Average High Temp
        temp_avghigh = temp_string.split(" | ")[4]
        temp_avghigh_list.append(int(temp_avghigh.split(" °F")[0]))

        # Average Low Temp
        temp_avglow = temp_string.split(" | ")[5]
        temp_avglow_list.append(int(temp_avglow.split(" °F")[0]))

        # Historic High Temp
        temp_high = temp_string.split(" | ")[6]
        temp_high = temp_high.split("high: ")[1]
        temp_high_list.append(int(temp_high.split(" °F")[0]))

#         # Temperature Range
#         average_temp_range_list.append(temp_avghigh - temp_avglow)
        
        print(f"Processing Record {row_count} of Set 1 | {city_query}, {state_query}")
        row_count += 1
        
    except Exception as e:
        print(f"{city_query} not found")
        row_count += 1
                       
print(avg_temp_clean)
print(temp_low_list)
print(temp_avghigh_list)
print(temp_avglow_list)
print(temp_high_list)
# print(average_temp_range_list)

Beginning Data Retrieval
------------------------
This is the city query Birmingham, Alabama
Processing Record 1 of Set 1 | Birmingham, Alabama
This is the city query Hoover, Alabama
Processing Record 2 of Set 1 | Hoover, Alabama
This is the city query Huntsville, Alabama
Processing Record 3 of Set 1 | Huntsville, Alabama
[63, 63, 59]
[11, 11, 5]
[83, 83, 81]
[42, 42, 37]
[104, 104, 102]
[]


# Census Data

In [8]:
from census import Census
from us import states
import gmaps

# Census & gmaps API Keys
from config import (census_api_key, gkey)
c = Census(census_api_key, year=2013)

# Configure gmaps
gmaps.configure(api_key=gkey)

In [6]:
# What is Available
# pprint(c.acs5.tables())

In [ ]:
# Figuring out CityFIPS
# US places
# (example: Fresno city in California is "27000", or prefixed with the state code "0627000")
# state_place(fields, state_fips, place)


# Address	2630 CAHABA RD, BIRMINGHAM, AL, 35223
# MSA/MD Code	13820
# State Code	01
# County Code	073
# Tract Code	0108.02
# MSA/MD Name	BIRMINGHAM-HOOVER, AL
# State Name	ALABAMA
# County Name	JEFFERSON COUNTY

# In the CSV = 0107000-01073010802
# so State code = 01, Place Code = 07000





# state_county_tract Specification
state_county_tract(fields, state_fips, county_fips, tract)

#The geographic name for all census tracts for county 170 in Alaska:
c.sf1.get('NAME', geo={'for': 'tract:*',
                       'in': 'state:{} county:170'.format(states.AK.fips)})

#The same call using the state_county_tract convenience method:
c.sf1.state_county_tract('NAME', states.AK.fips, '170', Census.ALL)




state_place(fields, state_fips, place)


In [10]:
census_data_test = c.acs5.state_place('NAME',states.AL.fips,'07000')
census_pd_test = pd.DataFrame(census_data_test)
census_data_test

[{'NAME': 'Birmingham city, Alabama', 'state': '01', 'place': '07000'}]

In [ ]:
# Run Census Search to retrieve data on all states
# Note the addition of "B23025_005E" for unemployment count
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
                          "B19301_001E",
                          "B17001_002E",
                          "B23025_005E"), {'for': 'state:*'})


# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",
                                      "B17001_002E": "Poverty Count",
                                      "B23025_005E": "Unemployment Count",
                                      "NAME": "Name", "state": "State"})

# Add in Poverty Rate (Poverty Count / Population)
census_pd["Poverty Rate"] = 100 * \
    census_pd["Poverty Count"].astype(
        int) / census_pd["Population"].astype(int)

# Add in Employment Rate (Employment Count / Population)
census_pd["Unemployment Rate"] = 100 * \
    census_pd["Unemployment Count"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Name", "Population", "Median Age", "Household Income",
                       "Per Capita Income", "Poverty Count", "Poverty Rate", "Unemployment Rate"]]

census_pd.head()